In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import flair
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np



/Users/amitobe/opt/anaconda3/envs/nlp_course/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [22]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
bert_model = AutoModelForSequenceClassification.from_pretrained(MODEL)
flair_model = flair.models.TextClassifier.load('en-sentiment')

def process_flair(dialogue):   # returning the flair score
    sentence = flair.data.Sentence(dialogue)
    flair_model.predict(sentence)
    label = sentence.labels[0].value
    score = sentence.labels[0].score
    if label == 'POSITIVE':
        return score
    elif label == 'NEGATIVE':
        return -score



def return_sentiment(txt):  # returning single-sentence BERT score
    encoded_input = tokenizer(txt, return_tensors='pt',padding=True,truncation=True)
    output = bert_model(**encoded_input)
    score = output[0][0].detach().numpy() 
    scores = softmax(score)
    if np.argsort(scores)[2] == 1:
        return 0
    else:
        return (np.argsort(scores)[2]-1)*scores[np.argsort(scores)[2]]
    
def tb_score(txt):
    sen = TextBlob(txt)
    return pd.Series({'tb': sen.sentiment.polarity})

def cal_vader_textblob_bert_flair(txt):
    tb_score = TextBlob(txt).sentiment.polarity
    obj = SentimentIntensityAnalyzer()
    vader_score = obj.polarity_scores(txt)['compound']
    flair_score = process_flair(txt)
    bert_score = return_sentiment(txt)
    #prob = logmodel3.predict_proba([[tb_score,vader_score,flair_score,bert_score]])[0]      
    return np.array([[vader_score, tb_score,bert_score,flair_score]])



In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
import joblib



In [9]:
# Custom transformer for a function
class FunctionTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, func):
        self.func = func
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return self.func(X)





In [52]:
# Load the trained model (replace 'trained_model.pkl' with your model file)
conversational_lr_loaded_model = joblib.load('conversational_lr_sav_model.pkl')



In [54]:
conversational_lr_loaded_model.predict([[-0.8256    , -0.65      , -0.83824599, -0.99770665]])

array([-1])

In [61]:
# Create a prediction pipeline with custom preprocessing and model
prediction_pipeline = Pipeline([
    ('custom_function', FunctionTransformer(func=cal_vader_textblob_bert_flair)),  # Custom preprocessing step
    #('scaler', StandardScaler()),  # Standard scaling (for new data)
   ('model', conversational_lr_loaded_model )  # Trained model (loaded)
])



In [62]:
new_df_final = [
    "This bad thing should never have happened like this, its quite sad",
    "Me too.",
    "What am I?",
    "I'm whatever"
]

In [63]:
cal_vader_textblob_bert_flair("I am very sad")

array([[-0.5256    , -0.65      , -0.83824599, -0.99770665]])

In [66]:
# Make predictions using the pipeline
predictions = prediction_pipeline.predict(new_df_final[0])

# Display the predictions
print("Predictions:", predictions)



Predictions: [-1]


In [59]:
new_df_final

['This bad thing should never have happened like this, its quite sad',
 'Me too.',
 'What am I?',
 "I'm whatever"]

In [67]:
from sklearn import set_config

In [68]:
set_config(display="diagram")

In [70]:
prediction_pipeline

Pipeline(steps=[('custom_function',
                 FunctionTransformer(func=<function cal_vader_textblob_bert_flair at 0x7ff2a8ac93b0>)),
                ('model', LogisticRegression())])